<h3>data/temperature/*.csv -> result/temperature/result.csv</h3>
<ol style="color: blue">
  <li>Remove empty lines </li>
  <li>Replace date to year </li>
  <li>Remove cells with older than 1990</li>
  <li>Groups by country+year and sum averages for temperatures</li>
</ol>

In [234]:
! cat data/temperature/*.csv | awk -f com/temperature/removeEmpty.awk | awk -f com/temperature/replaceYearDate.awk | awk -f com/temperature/filterYears.awk | awk -f com/temperature/groupByYearAverageSum.awk  > result/temperature/grouped.csv; awk -f com/countColumsRows.awk result/temperature/grouped.csv 

Columns: 4
Rows: 5785


<h3>data/energy/*.csv -> result/energy/result.csv</h3>
<ol style="color: blue">
  <li>Merge all countries </li>
  <li>Remove cells with older than 1990</li>
</ol>

In [319]:
! awk -f com/energy/merge.awk data/energy/*.csv | awk -f com/energy/filterYears.awk  > result/energy/merged.csv; awk -f com/countColumsRows.awk result/energy/merged.csv

Columns: 128
Rows: 7292


<h3>result/temperature/grouped.csv -> temperature_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 4(country) And 1(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [328]:
!cat result/temperature/grouped.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $4.$1' > temperature_key.csv; awk -f com/countColumsRows.awk temperature_key.csv

Columns: 5
Rows: 5785


<h3>result/energy/merged.csv -> energy_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 2(country) And 3(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [329]:
!cat result/energy/merged.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $2.$3' > energy_key.csv; awk -f com/countColumsRows.awk energy_key.csv 

Columns: 129
Rows: 7292


<h3>temperature_key.csv  energy_key.csv -> result/temperature_energy_joined.csv</h3>
<ol style="color: blue">
  <li>Join by key column</li>
  <li>Removes key column</li>
</ol>

In [330]:
! mlr --csv join -j key -f temperature_key.csv energy_key.csv | awk -F , 'BEGIN {OFS=FS}  {$1=""; sub(",", ""); print}' > result/temperature_energy_joined.csv; rm temperature_key.csv energy_key.csv; awk -f com/countColumsRows.awk result/temperature_energy_joined.csv

Columns: 132
Rows: 4460


<h3>Select columns and rows</h3>
OFS="[output separator]" { if ([column index] == "Russia") print [column index], ...}

In [375]:
!awk -F, 'OFS=","{if ($4 == "Russia") print $1, $2}' result/temperature_energy_joined.csv 

1990,-4.02292
1991,-4.237
1992,-4.90325
1993,-4.62175
1994,-4.89842
1995,-3.13017
1996,-4.81033
1997,-4.2965
1998,-5.28108
1999,-4.82975
2000,-4.54567
2001,-4.49067
2002,-4.12292
2003,-3.83283
2004,-4.49358
2005,-3.49808
2006,-4.763
2007,-2.97508
2008,-3.30542
2009,-4.6085
2010,-4.52058
2011,-3.40158
2012,-3.90175
2013,-2.26313


In [ ]:
!mlr --icsv --opprint cut -f country,year,coal_prod_change_pct result/temperature_energy_joined.csv